# Automated Sentinel-2 Multi-City Processing Pipeline
## Create 21-band Multi-Month Stacks for Multiple Cities

**Cities:** Vienna, Warsaw, Vancouver, Toronto

**Months:** April, August, November

**Output:** 4 cities × 21 bands (4 spectral + 3 indices per month × 3 months)

## 1. Import Libraries

In [ ]:
import os
import glob
import numpy as np
import geopandas as gpd
import rioxarray as rxr
import xarray as xr
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

## 2. Configuration

In [ ]:
# Base paths
BASE_PATH = "/Users/timgotschim/Documents/InfraredTIM/INFRARED"
GEOJSON_FOLDER = os.path.join(BASE_PATH, "aois_json")
OUTPUT_FOLDER = os.path.join(BASE_PATH, "processed_stacks")
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Cities to process
CITIES = ["Wien", "Warsaw", "Vancouver", "Toronto"]

# Months and their folder names (adjust these names based on your folder structure)
MONTHS = ["Apr", "Aug", "Nov"]

# Sentinel-2 bands to process (10m resolution)
BANDS = ["B02", "B03", "B04", "B08"]

print("Configuration:")
print(f"  Base path: {BASE_PATH}")
print(f"  Cities: {', '.join(CITIES)}")
print(f"  Months: {', '.join(MONTHS)}")
print(f"  Bands: {', '.join(BANDS)}")
print(f"  Output folder: {OUTPUT_FOLDER}")

## 3. Helper Functions

In [ ]:
def find_sentinel_folder(base_path, city, month):
    """
    Find the Sentinel-2 folder for a given city and month.
    Handles flexible folder naming patterns.
    """
    # Try different naming patterns
    patterns = [
        f"{city} {month}",
        f"{city}_{month}",
        f"{city.lower()} {month.lower()}",
        f"{city.lower()}_{month.lower()}"
    ]
    
    for pattern in patterns:
        search_path = os.path.join(base_path, pattern)
        if os.path.exists(search_path):
            # Find the R10m folder inside
            r10m_folders = glob.glob(os.path.join(search_path, "**/R10m"), recursive=True)
            if r10m_folders:
                return r10m_folders[0]
    
    return None


def find_geojson(geojson_folder, city):
    """
    Find the GeoJSON file for a given city.
    """
    # Try different naming patterns
    patterns = [
        f"{city}.geojson",
        f"{city.lower()}.geojson",
        f"{city.upper()}.geojson"
    ]
    
    for pattern in patterns:
        geojson_path = os.path.join(geojson_folder, pattern)
        if os.path.exists(geojson_path):
            return geojson_path
    
    return None


def load_and_clip_band(band_folder, band_name, aoi_geometry):
    """
    Load a Sentinel-2 band and clip it to the AOI.
    """
    # Find band file
    band_files = glob.glob(os.path.join(band_folder, f"*{band_name}_10m.jp2"))
    
    if not band_files:
        print(f"    WARNING: Band {band_name} not found in {band_folder}")
        return None
    
    band_file = band_files[0]
    
    try:
        # Load and clip
        band = rxr.open_rasterio(band_file, masked=True).squeeze()
        band_clipped = band.rio.clip([aoi_geometry], crs="EPSG:4326")
        return band_clipped
    except Exception as e:
        print(f"    ERROR loading {band_name}: {e}")
        return None


def calculate_vegetation_indices(bands_dict):
    """
    Calculate NDVI, EVI, and SAVI from spectral bands.
    """
    indices = {}
    
    if "B08" in bands_dict and "B04" in bands_dict:
        nir = bands_dict["B08"].astype(np.float32)
        red = bands_dict["B04"].astype(np.float32)
        
        # NDVI: (NIR - Red) / (NIR + Red)
        ndvi = (nir - red) / (nir + red)
        ndvi = xr.where(np.isfinite(ndvi), ndvi, np.nan)
        indices["NDVI"] = ndvi
        
        # SAVI: ((NIR - Red) * (1 + L)) / (NIR + Red + L), L=0.5
        L = 0.5
        savi = ((nir - red) * (1 + L)) / (nir + red + L)
        savi = xr.where(np.isfinite(savi), savi, np.nan)
        indices["SAVI"] = savi
        
        # EVI: 2.5 * (NIR - Red) / (NIR + 6*Red - 7.5*Blue + 1)
        if "B02" in bands_dict:
            blue = bands_dict["B02"].astype(np.float32)
            evi = 2.5 * (nir - red) / (nir + 6*red - 7.5*blue + 1)
            evi = xr.where(np.isfinite(evi), evi, np.nan)
            indices["EVI"] = evi
    
    return indices


print("✓ Helper functions defined")

## 4. Process All Cities
### Main processing loop

In [ ]:
print("="*80)
print("STARTING MULTI-CITY PROCESSING")
print("="*80)

processing_summary = []

for city in CITIES:
    print(f"\n{'='*80}")
    print(f"PROCESSING: {city.upper()}")
    print(f"{'='*80}")
    
    # Find GeoJSON file
    geojson_file = find_geojson(GEOJSON_FOLDER, city)
    
    if not geojson_file:
        print(f"✗ GeoJSON not found for {city}")
        processing_summary.append({"city": city, "status": "FAILED", "reason": "GeoJSON not found"})
        continue
    
    print(f"✓ Found GeoJSON: {geojson_file}")
    
    # Load AOI
    try:
        aoi = gpd.read_file(geojson_file)
        if aoi.crs is None:
            aoi.set_crs("EPSG:4326", inplace=True)
        if aoi.crs.to_epsg() != 4326:
            aoi = aoi.to_crs("EPSG:4326")
        
        aoi_geometry = aoi.geometry.iloc[0]
        print(f"✓ AOI loaded: bounds {aoi.total_bounds}")
    except Exception as e:
        print(f"✗ Error loading AOI: {e}")
        processing_summary.append({"city": city, "status": "FAILED", "reason": f"AOI load error: {e}"})
        continue
    
    # Storage for all bands
    all_band_arrays = []
    all_band_names = []
    
    # Process each month
    for month in MONTHS:
        print(f"\n  --- Processing {month} ---")
        
        # Find Sentinel-2 folder
        sentinel_folder = find_sentinel_folder(GEOJSON_FOLDER, city, month)
        
        if not sentinel_folder:
            print(f"  ✗ Sentinel-2 data not found for {city} {month}")
            continue
        
        print(f"  ✓ Found Sentinel folder: {sentinel_folder}")
        
        # Load spectral bands
        month_bands = {}
        
        for band in BANDS:
            print(f"    Loading {band}...", end=" ")
            band_data = load_and_clip_band(sentinel_folder, band, aoi_geometry)
            
            if band_data is not None:
                band_name = f"{band}-{month}"
                all_band_arrays.append(band_data)
                all_band_names.append(band_name)
                month_bands[band] = band_data
                print(f"✓ shape: {band_data.shape}")
            else:
                print("✗ failed")
        
        # Calculate vegetation indices
        if len(month_bands) >= 3:
            print(f"    Calculating vegetation indices...")
            indices = calculate_vegetation_indices(month_bands)
            
            for index_name, index_data in indices.items():
                full_name = f"{index_name}-{month}"
                all_band_arrays.append(index_data)
                all_band_names.append(full_name)
                print(f"      ✓ {index_name}: range [{float(index_data.min()):.3f}, {float(index_data.max()):.3f}]")
    
    # Create final stack
    if len(all_band_arrays) == 0:
        print(f"\n✗ No bands processed for {city}")
        processing_summary.append({"city": city, "status": "FAILED", "reason": "No bands processed"})
        continue
    
    print(f"\n  --- Creating final stack ---")
    
    try:
        # Stack all bands
        stack = xr.concat(all_band_arrays, dim="band")
        stack = stack.assign_coords(band=all_band_names)
        stack = stack.astype(np.float32)
        
        print(f"  ✓ Stacked shape: {stack.shape}")
        print(f"  ✓ Total bands: {len(all_band_names)}")
        print(f"  ✓ Band names: {all_band_names}")
        
        # Save stack
        output_file = os.path.join(OUTPUT_FOLDER, f"{city}_MultiMonth_stack.tif")
        stack.rio.to_raster(output_file, dtype=np.float32)
        
        print(f"\n✓ SAVED: {output_file}")
        processing_summary.append({
            "city": city, 
            "status": "SUCCESS", 
            "bands": len(all_band_names),
            "output": output_file
        })
        
    except Exception as e:
        print(f"\n✗ Error creating stack: {e}")
        processing_summary.append({"city": city, "status": "FAILED", "reason": f"Stack creation error: {e}"})

print(f"\n{'='*80}")
print("PROCESSING COMPLETE")
print(f"{'='*80}")

## 5. Processing Summary

In [ ]:
print("\n" + "="*80)
print("PROCESSING SUMMARY")
print("="*80)

success_count = 0
failed_count = 0

for result in processing_summary:
    city = result["city"]
    status = result["status"]
    
    if status == "SUCCESS":
        success_count += 1
        print(f"\n✓ {city}:")
        print(f"  Status: SUCCESS")
        print(f"  Bands: {result['bands']}")
        print(f"  Output: {result['output']}")
    else:
        failed_count += 1
        print(f"\n✗ {city}:")
        print(f"  Status: FAILED")
        print(f"  Reason: {result['reason']}")

print(f"\n{'-'*80}")
print(f"Total cities processed: {len(processing_summary)}")
print(f"  Successful: {success_count}")
print(f"  Failed: {failed_count}")
print("="*80)

if success_count > 0:
    print(f"\n✓ Output folder: {OUTPUT_FOLDER}")
    print("\nGenerated files:")
    for result in processing_summary:
        if result["status"] == "SUCCESS":
            print(f"  - {os.path.basename(result['output'])}")

## 6. Verify Output Files

In [ ]:
import rasterio

print("\n" + "="*80)
print("VERIFYING OUTPUT FILES")
print("="*80)

for result in processing_summary:
    if result["status"] == "SUCCESS":
        output_file = result["output"]
        city = result["city"]
        
        print(f"\n{city}:")
        print(f"  File: {os.path.basename(output_file)}")
        
        try:
            with rasterio.open(output_file) as src:
                print(f"  Bands: {src.count}")
                print(f"  Shape: {src.height} x {src.width} pixels")
                print(f"  CRS: {src.crs}")
                print(f"  Resolution: {src.res[0]:.2f} m")
                print(f"  Bounds: {src.bounds}")
                print(f"  File size: {os.path.getsize(output_file) / (1024**2):.2f} MB")
        except Exception as e:
            print(f"  ERROR verifying file: {e}")

print("\n" + "="*80)

## 7. Expected Band Structure
### Each stack should have 21 bands in this order:

In [ ]:
expected_bands = [
    # April (7 bands)
    "B02-Apr", "B03-Apr", "B04-Apr", "B08-Apr",
    "NDVI-Apr", "EVI-Apr", "SAVI-Apr",
    
    # August (7 bands)
    "B02-Aug", "B03-Aug", "B04-Aug", "B08-Aug",
    "NDVI-Aug", "EVI-Aug", "SAVI-Aug",
    
    # November (7 bands)
    "B02-Nov", "B03-Nov", "B04-Nov", "B08-Nov",
    "NDVI-Nov", "EVI-Nov", "SAVI-Nov"
]

print("Expected Band Structure (21 bands total):")
print("="*80)
for i, band in enumerate(expected_bands, 1):
    print(f"  {i:2d}. {band}")
print("="*80)

print("\nSpectral Bands:")
print("  B02 - Blue (490 nm)")
print("  B03 - Green (560 nm)")
print("  B04 - Red (665 nm)")
print("  B08 - NIR (842 nm)")

print("\nVegetation Indices:")
print("  NDVI - Normalized Difference Vegetation Index")
print("  EVI  - Enhanced Vegetation Index")
print("  SAVI - Soil Adjusted Vegetation Index")